In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('sqlite_database.db')  # Conecte no banco


In [2]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tabelas = pd.read_sql_query(query, conn)
print(tabelas)


            name
0    staging_dre
1    dim_unidade
2  dim_estrutura
3       fato_dre


In [6]:
# 1. Converter MesRef para o formato 'YYYY-MM'
import pandas as pd

# Atualizar o formato em staging_dre
query = "SELECT *, strftime('%Y-%m', MesRef / 1000, 'unixepoch') AS MesRefFormatado FROM staging_dre;"
staging_dre = pd.read_sql_query(query, conn)

# Atualizar o formato em fato_dre
query = "SELECT *, strftime('%Y-%m', MesRef / 1000, 'unixepoch') AS MesRefFormatado FROM fato_dre;"
fato_dre = pd.read_sql_query(query, conn)

# Visualizar os resultados
print("Staging DRE - Datas formatadas:")
print(staging_dre[['MesRef', 'MesRefFormatado']].drop_duplicates())

print("\nFato DRE - Datas formatadas:")
print(fato_dre[['MesRef', 'MesRefFormatado']].drop_duplicates())


Staging DRE - Datas formatadas:
             MesRef MesRefFormatado
0     1677639600000         2023-03
1296  1680318000000         2023-04
2592  1682910000000         2023-05
3888  1685588400000         2023-06

Fato DRE - Datas formatadas:
          MesRef MesRefFormatado
0  1677639600000         2023-03
1  1680318000000         2023-04
2  1682910000000         2023-05
3  1685588400000         2023-06


In [7]:
import re

# Extrair códigos das unidades no staging_dre
def extrair_codigo(unidade):
    if pd.isnull(unidade) or "Total" in unidade:
        return 0  # Código para Total
    match = re.search(r'\d+$', unidade)  # Capturar o código no final do texto
    return int(match.group()) if match else 9999111  # Código padrão para escritório

# Aplicar a função
staging_dre['CodigoUnidade'] = staging_dre['Unidade'].apply(extrair_codigo)

# Comparar com a dim_unidade
dim_unidade = pd.read_sql_query("SELECT * FROM dim_unidade;", conn)
codigos_dim = dim_unidade['CodigoUnidade'].unique()
codigos_staging = staging_dre['CodigoUnidade'].unique()

# Verificar códigos faltantes
codigos_faltantes = set(codigos_staging) - set(codigos_dim)

# Exibir os resultados
print("Códigos no Staging:", codigos_staging)
print("Códigos na Dimensão:", codigos_dim)
print("Códigos Faltantes:", codigos_faltantes)

# Mostrar as primeiras linhas para validação
print("\nStaging DRE com códigos ajustados:")
print(staging_dre[['Unidade', 'CodigoUnidade']].drop_duplicates())


Códigos no Staging: [9999111   11111   11113   11116   11117]
Códigos na Dimensão: [9999111   11111   11112   11113   11114   11115   11116   11117]
Códigos Faltantes: set()

Staging DRE com códigos ajustados:
                       Unidade  CodigoUnidade
0         9999111 - ESCRITÓRIO        9999111
1            Unidade 1 - 11111          11111
2  Unidade 2 - 11112 - Atacado        9999111
3            Unidade 3 - 11113          11113
4  Unidade 4 - 11114 - Atacado        9999111
5  Unidade 5 - 11115 - Atacado        9999111
6            Unidade 6 - 11116          11116
7            Unidade 7 - 11117          11117


In [8]:
# Correção específica para os códigos mal atribuídos
correcoes = {
    'Unidade 2 - 11112 - Atacado': 11112,
    'Unidade 4 - 11114 - Atacado': 11114,
    'Unidade 5 - 11115 - Atacado': 11115
}

# Aplicar correções diretamente
staging_dre['CodigoUnidade'] = staging_dre.apply(
    lambda row: correcoes.get(row['Unidade'], row['CodigoUnidade']), axis=1
)

# Verificar novamente os códigos após a correção
codigos_staging_corrigidos = staging_dre['CodigoUnidade'].unique()
codigos_faltantes_corrigidos = set(codigos_staging_corrigidos) - set(codigos_dim)

# Exibir resultados
print("Códigos no Staging após correção:", codigos_staging_corrigidos)
print("Códigos faltantes após correção:", codigos_faltantes_corrigidos)

# Mostrar as primeiras linhas atualizadas
print("\nStaging DRE com códigos corrigidos:")
print(staging_dre[['Unidade', 'CodigoUnidade']].drop_duplicates())


Códigos no Staging após correção: [9999111   11111   11112   11113   11114   11115   11116   11117]
Códigos faltantes após correção: set()

Staging DRE com códigos corrigidos:
                       Unidade  CodigoUnidade
0         9999111 - ESCRITÓRIO        9999111
1            Unidade 1 - 11111          11111
2  Unidade 2 - 11112 - Atacado          11112
3            Unidade 3 - 11113          11113
4  Unidade 4 - 11114 - Atacado          11114
5  Unidade 5 - 11115 - Atacado          11115
6            Unidade 6 - 11116          11116
7            Unidade 7 - 11117          11117


In [13]:
# Carregar a tabela dim_estrutura do banco
query = "SELECT * FROM dim_estrutura;"
dim_estrutura = pd.read_sql_query(query, conn)





In [16]:
# Mesclar staging_dre com dim_estrutura para trazer o IdEstrutura
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar a mesclagem
print(staging_dre[['LinhaEstrutura', 'IdEstrutura']].head(10))


                  LinhaEstrutura  IdEstrutura
0  1 - Receita Operacional Bruta          1.0
1  1 - Receita Operacional Bruta          1.0
2  1 - Receita Operacional Bruta          1.0
3  1 - Receita Operacional Bruta          1.0
4  1 - Receita Operacional Bruta          1.0
5  1 - Receita Operacional Bruta          1.0
6  1 - Receita Operacional Bruta          1.0
7  1 - Receita Operacional Bruta          1.0
8    1.1 - Vendas de mercadorias          2.0
9    1.1 - Vendas de mercadorias          2.0


In [17]:
# Mesclar staging_dre com dim_unidade para adicionar detalhes das unidades
staging_dre = staging_dre.merge(
    dim_unidade[['CodigoUnidade', 'IdUnidade']],
    on='CodigoUnidade',
    how='left'
)

# Validar a mesclagem
print(staging_dre[['CodigoUnidade', 'IdUnidade']].drop_duplicates())


   CodigoUnidade  IdUnidade
0        9999111          1
1          11111          2
2          11112          3
3          11113          4
4          11114          5
5          11115          6
6          11116          7
7          11117          8


In [19]:
# Verificar linhas com NaN em IdEstrutura
problemas_estrutura = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(problemas_estrutura[['LinhaEstrutura', 'MesRef', 'Unidade']])


                                         LinhaEstrutura         MesRef  \
96              4.1 - Custo da Mercadoria Vendida - CMV  1677639600000   
97              4.1 - Custo da Mercadoria Vendida - CMV  1677639600000   
98              4.1 - Custo da Mercadoria Vendida - CMV  1677639600000   
99              4.1 - Custo da Mercadoria Vendida - CMV  1677639600000   
100             4.1 - Custo da Mercadoria Vendida - CMV  1677639600000   
...                                                 ...            ...   
5267  14 - (+/-) Outras Receitas e Despesas não Oper...  1685588400000   
5268  14 - (+/-) Outras Receitas e Despesas não Oper...  1685588400000   
5269  14 - (+/-) Outras Receitas e Despesas não Oper...  1685588400000   
5270  14 - (+/-) Outras Receitas e Despesas não Oper...  1685588400000   
5271  14 - (+/-) Outras Receitas e Despesas não Oper...  1685588400000   

                          Unidade  
96           9999111 - ESCRITÓRIO  
97              Unidade 1 - 11111  
98 

In [20]:
# Encontrar diferenças entre os dois datasets
estruturas_staging = staging_dre['LinhaEstrutura'].unique()
estruturas_dim = dim_estrutura['LinhaEstrutura'].unique()

# Mostrar as diferenças
diferencas = set(estruturas_staging) - set(estruturas_dim)
print(diferencas)


{'11 - Resultado Operacional -Ebitda', '8.6.5 - Multa - Veículos', '14 - (+/-) Outras Receitas e Despesas não Operacionais', '4.1 - Custo da Mercadoria Vendida - CMV', '8.8.2.1 - Doações - Outras entidades', '4.2 - Custo dos Serviços Prestados - CSP', '12 - (+/-) Resultado Financeiro Líquido'}


In [21]:
# Adicionar nova estrutura faltante
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 200,  # Novo ID
    'LinhaEstrutura': '11 - Resultado Operacional -Ebitda',
    'LinhaRelatorio': 'Resultado Operacional - Ebitda',
    'TipoLinha': 'Grupo',
    'GrupoDaConta': '(=) Resultado Operacional - Ebitda',
    'TipoEstrutura': 'Resultado',
    'Int1': 11,
    'Int2': None,
    'Int3': None,
    'Int4': None,
    'Subgrupo': None,
    'Subgrupo2': None,
    'IdEstrutura_sukey_pai': None,
    'SomaDe': None
}])

# Concatenar com a dimensão existente
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Validar inclusão
print(dim_estrutura[dim_estrutura['LinhaEstrutura'] == '11 - Resultado Operacional -Ebitda'])


     IdEstrutura IdEstrutura_sukey                      LinhaEstrutura  \
163          200               NaN  11 - Resultado Operacional -Ebitda   

                     LinhaRelatorio TipoLinha  \
163  Resultado Operacional - Ebitda     Grupo   

                           GrupoDaConta TipoEstrutura  Int1  Int2  Int3  Int4  \
163  (=) Resultado Operacional - Ebitda     Resultado    11   NaN   NaN   NaN   

    Subgrupo Subgrupo2 IdEstrutura_sukey_pai  SomaDe  
163     None      None                  None     NaN  


<ipython-input-21-4e1bf45ac4ce>:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [23]:
# Mesclagem corrigida evitando duplicatas
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove a coluna duplicada antes
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar resultado
faltantes_pos_correcao = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_correcao['LinhaEstrutura'].unique())


['4.1 - Custo da Mercadoria Vendida - CMV'
 '4.2 - Custo dos Serviços Prestados - CSP' '8.6.5 - Multa - Veículos'
 '8.8.2.1 - Doações - Outras entidades'
 '12 - (+/-) Resultado Financeiro Líquido'
 '14 - (+/-) Outras Receitas e Despesas não Operacionais']


In [24]:
# Adicionar estrutura para '4.1 - Custo da Mercadoria Vendida - CMV'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 201,
    'IdEstrutura_sukey': '4.1',
    'LinhaEstrutura': '4.1 - Custo da Mercadoria Vendida - CMV',
    'LinhaRelatorio': 'Custo da Mercadoria Vendida',
    'TipoLinha': 'Conta',
    'GrupoDaConta': '(-) Custo das Vendas',
    'TipoEstrutura': 'Despesa',
    'Int1': 4,
    'Int2': 1,
    'Int3': None,
    'Int4': None,
    'Subgrupo': '4.1 - CMV',
    'Subgrupo2': None,
    'IdEstrutura_sukey_pai': '4',
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_1 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_1['LinhaEstrutura'].unique())


['4.2 - Custo dos Serviços Prestados - CSP' '8.6.5 - Multa - Veículos'
 '8.8.2.1 - Doações - Outras entidades'
 '12 - (+/-) Resultado Financeiro Líquido'
 '14 - (+/-) Outras Receitas e Despesas não Operacionais']


<ipython-input-24-500d96444bb8>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [25]:
# Adicionar estrutura para '4.2 - Custo dos Serviços Prestados - CSP'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 202,
    'IdEstrutura_sukey': '4.2',
    'LinhaEstrutura': '4.2 - Custo dos Serviços Prestados - CSP',
    'LinhaRelatorio': 'Custo dos Serviços Prestados',
    'TipoLinha': 'Conta',
    'GrupoDaConta': '(-) Custo das Vendas',
    'TipoEstrutura': 'Despesa',
    'Int1': 4,
    'Int2': 2,
    'Int3': None,
    'Int4': None,
    'Subgrupo': '4.2 - CSP',
    'Subgrupo2': None,
    'IdEstrutura_sukey_pai': '4',
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_2 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_2['LinhaEstrutura'].unique())


['8.6.5 - Multa - Veículos' '8.8.2.1 - Doações - Outras entidades'
 '12 - (+/-) Resultado Financeiro Líquido'
 '14 - (+/-) Outras Receitas e Despesas não Operacionais']


<ipython-input-25-5b527bc16751>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [26]:
# Adicionar estrutura para '8.6.5 - Multa - Veículos'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 865,
    'IdEstrutura_sukey': '8.6.5',
    'LinhaEstrutura': '8.6.5 - Multa - Veículos',
    'LinhaRelatorio': 'Multa - Veículos',
    'TipoLinha': 'Conta',
    'GrupoDaConta': '(-) Despesas Operacionais',
    'TipoEstrutura': 'Despesa',
    'Int1': 8,
    'Int2': 6,
    'Int3': 5,
    'Int4': None,
    'Subgrupo': '8.6 - Despesas Gerais',
    'Subgrupo2': '8.6.5 - Multa - Veículos',
    'IdEstrutura_sukey_pai': '8.6',
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_3 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_3['LinhaEstrutura'].unique())


['8.8.2.1 - Doações - Outras entidades'
 '12 - (+/-) Resultado Financeiro Líquido'
 '14 - (+/-) Outras Receitas e Despesas não Operacionais']


<ipython-input-26-9d52c4d83d3d>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [27]:
# Adicionar estrutura para '8.8.2.1 - Doações - Outras entidades'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 8821,
    'IdEstrutura_sukey': '8.8.2.1',
    'LinhaEstrutura': '8.8.2.1 - Doações - Outras entidades',
    'LinhaRelatorio': 'Doações - Outras entidades',
    'TipoLinha': 'Conta',
    'GrupoDaConta': '(-) Despesas Operacionais',
    'TipoEstrutura': 'Despesa',
    'Int1': 8,
    'Int2': 8,
    'Int3': 2,
    'Int4': 1,
    'Subgrupo': '8.8 - Doações',
    'Subgrupo2': '8.8.2.1 - Doações - Outras entidades',
    'IdEstrutura_sukey_pai': '8.8.2',
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_4 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_4['LinhaEstrutura'].unique())


['12 - (+/-) Resultado Financeiro Líquido'
 '14 - (+/-) Outras Receitas e Despesas não Operacionais']


<ipython-input-27-f9ad232987a3>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [28]:
# Adicionar estrutura para '12 - (+/-) Resultado Financeiro Líquido'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 1200,
    'IdEstrutura_sukey': '12',
    'LinhaEstrutura': '12 - (+/-) Resultado Financeiro Líquido',
    'LinhaRelatorio': 'Resultado Financeiro Líquido',
    'TipoLinha': 'Grupo',
    'GrupoDaConta': '(+/-) Resultado Financeiro Líquido',
    'TipoEstrutura': 'Resultado',
    'Int1': 12,
    'Int2': None,
    'Int3': None,
    'Int4': None,
    'Subgrupo': None,
    'Subgrupo2': None,
    'IdEstrutura_sukey_pai': None,
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_5 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_5['LinhaEstrutura'].unique())


['14 - (+/-) Outras Receitas e Despesas não Operacionais']


<ipython-input-28-a10536999a2c>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [29]:
# Adicionar estrutura para '14 - (+/-) Outras Receitas e Despesas não Operacionais'
nova_estrutura = pd.DataFrame([{
    'IdEstrutura': 1400,
    'IdEstrutura_sukey': '14',
    'LinhaEstrutura': '14 - (+/-) Outras Receitas e Despesas não Operacionais',
    'LinhaRelatorio': 'Outras Receitas e Despesas não Operacionais',
    'TipoLinha': 'Grupo',
    'GrupoDaConta': '(+/-) Outras Receitas e Despesas não Operacionais',
    'TipoEstrutura': 'Resultado',
    'Int1': 14,
    'Int2': None,
    'Int3': None,
    'Int4': None,
    'Subgrupo': None,
    'Subgrupo2': None,
    'IdEstrutura_sukey_pai': None,
    'SomaDe': None
}])

# Concatenar no dim_estrutura
dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)

# Reexecutar a mesclagem
staging_dre = staging_dre.drop(columns=['IdEstrutura'], errors='ignore')  # Remove duplicatas
staging_dre = staging_dre.merge(
    dim_estrutura[['IdEstrutura', 'LinhaEstrutura']],
    on='LinhaEstrutura',
    how='left'
)

# Validar se ainda existem faltantes
faltantes_pos_6 = staging_dre[staging_dre['IdEstrutura'].isnull()]
print(faltantes_pos_6['LinhaEstrutura'].unique())


[]


<ipython-input-29-25fcded853a3>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_estrutura = pd.concat([dim_estrutura, nova_estrutura], ignore_index=True)


In [31]:
import numpy as np

# 1. Criar a tabela fato_dre_final consolidada
fato_dre_final = staging_dre[['IdEstrutura', 'CodigoUnidade', 'MesRef', 'Valor']].copy()

# 2. Renomear colunas para manter consistência
fato_dre_final.rename(columns={'CodigoUnidade': 'IdUnidade'}, inplace=True)

# 3. Corrigir valores nulos
fato_dre_final['IdEstrutura'] = fato_dre_final['IdEstrutura'].fillna(0).astype(int)
fato_dre_final['IdUnidade'] = fato_dre_final['IdUnidade'].fillna(0).astype(int)
fato_dre_final['MesRef'] = pd.to_datetime(fato_dre_final['MesRef'], unit='ms').dt.strftime('%Y-%m')
fato_dre_final['Valor'] = fato_dre_final['Valor'].fillna(0).astype(float)

# 4. Validar a estrutura consolidada
print(fato_dre_final.head(10))
print(fato_dre_final.dtypes)
print(fato_dre_final.isnull().sum())


   IdEstrutura  IdUnidade   MesRef       Valor
0            1    9999111  2023-03        0.00
1            1      11111  2023-03    63205.51
2            1      11112  2023-03   125431.88
3            1      11113  2023-03    60499.02
4            1      11114  2023-03   185317.24
5            1      11115  2023-03   297285.23
6            1      11116  2023-03   112564.40
7            1      11117  2023-03  1480406.53
8            2    9999111  2023-03        0.00
9            2      11111  2023-03    63205.51
IdEstrutura      int64
IdUnidade        int64
MesRef          object
Valor          float64
dtype: object
IdEstrutura    0
IdUnidade      0
MesRef         0
Valor          0
dtype: int64


In [34]:
import pandas as pd

# Caminhos corrigidos para os arquivos
dre_maio = pd.read_excel('/content/DRE_Real_05-2023.xlsx')
dre_junho = pd.read_excel('/content/DRE_Real_06-2023.xlsx')

# Exibir as primeiras linhas para validação
print("Maio 2023:")
print(dre_maio.head())

print("\nJunho 2023:")
print(dre_junho.head())


Maio 2023:
                                        Nome  9999111 - ESCRITÓRIO  \
0              1 - Receita Operacional Bruta                   0.0   
1                1.1 - Vendas de mercadorias                   0.0   
2                1.2 - Prestação de Serviços                   0.0   
3  2 - Deduções da Receita Operacional Bruta                   0.0   
4     2.1 - Impostos Incidentes sobre Vendas                   0.0   

   Unidade 1 - 11111  Unidade 2 - 11112 - Atacado  Unidade 3 - 11113  \
0           85027.69                    192090.14           96097.13   
1           85027.69                    192090.14           96097.13   
2               0.00                         0.00               0.00   
3             -92.13                      -106.26            -251.64   
4             -92.13                      -106.26            -251.64   

   Unidade 4 - 11114 - Atacado  Unidade 5 - 11115 - Atacado  \
0                    231129.26                    449435.86   
1        

In [36]:
import pandas as pd

# Carregar os arquivos de DRE
dre_marco = pd.read_excel('/content/DRE_Real_03-2023.xlsx')
dre_abril = pd.read_excel('/content/DRE_Real_04-2023.xlsx')
dre_maio = pd.read_excel('/content/DRE_Real_05-2023.xlsx')
dre_junho = pd.read_excel('/content/DRE_Real_06-2023.xlsx')

# Função para processar cada mês
def processar_dre(df, mes_ref):
    df_melted = df.melt(
        id_vars=['Nome'],
        var_name='Unidade',
        value_name='Valor'
    )
    df_melted['MesRef'] = mes_ref
    return df_melted

# Processar os meses individualmente
dre_marco_melted = processar_dre(dre_marco, '2023-03')
dre_abril_melted = processar_dre(dre_abril, '2023-04')
dre_maio_melted = processar_dre(dre_maio, '2023-05')
dre_junho_melted = processar_dre(dre_junho, '2023-06')

# Concatenar todos os meses
dre_totais = pd.concat([dre_marco_melted, dre_abril_melted, dre_maio_melted, dre_junho_melted], ignore_index=True)

# Criar códigos de unidade
dre_totais['CodigoUnidade'] = dre_totais['Unidade'].str.extract('(\d+)$')
dre_totais['CodigoUnidade'] = dre_totais['CodigoUnidade'].fillna(0).astype(int)

# Exibir resultado
print(dre_totais.head(10))


                                        Nome               Unidade  Valor  \
0              1 - Receita Operacional Bruta  9999111 - ESCRITÓRIO    0.0   
1                1.1 - Vendas de mercadorias  9999111 - ESCRITÓRIO    0.0   
2                1.2 - Prestação de Serviços  9999111 - ESCRITÓRIO    0.0   
3  2 - Deduções da Receita Operacional Bruta  9999111 - ESCRITÓRIO    0.0   
4     2.1 - Impostos Incidentes sobre Vendas  9999111 - ESCRITÓRIO    0.0   
5         2.1.1 - COFINS sobre Receita Bruta  9999111 - ESCRITÓRIO    0.0   
6            2.1.2 - PIS sobre Receita Bruta  9999111 - ESCRITÓRIO    0.0   
7           2.1.3 - ICMS sobre Receita Bruta  9999111 - ESCRITÓRIO    0.0   
8                   2.1.4 - Simples Nacional  9999111 - ESCRITÓRIO    0.0   
9                 2.2 - Devoluções de Vendas  9999111 - ESCRITÓRIO    0.0   

    MesRef  CodigoUnidade  
0  2023-03              0  
1  2023-03              0  
2  2023-03              0  
3  2023-03              0  
4  2023-03  

In [37]:
# Resumo por unidade e mês
resumo = dre_totais.groupby(['MesRef', 'CodigoUnidade'])['Valor'].sum().reset_index()

# Ordenar por mês e código da unidade
resumo = resumo.sort_values(by=['MesRef', 'CodigoUnidade'])

print(resumo)


     MesRef  CodigoUnidade       Valor
0   2023-03              0  3947137.85
1   2023-03          11111   112810.42
2   2023-03          11113   155887.64
3   2023-03          11116   332136.35
4   2023-03          11117  2226781.78
5   2023-04              0  6172323.18
6   2023-04          11111    56192.97
7   2023-04          11113   166398.78
8   2023-04          11116   391750.26
9   2023-04          11117  4215583.87
10  2023-05              0  9374836.62
11  2023-05          11111   214219.36
12  2023-05          11113   237780.94
13  2023-05          11116   577093.88
14  2023-05          11117  4917300.32
15  2023-06              0  5693319.49
16  2023-06          11111   113075.62
17  2023-06          11113   116338.98
18  2023-06          11116   410965.09
19  2023-06          11117  2969704.88


In [38]:
# Corrigir código do escritório
dre_totais.loc[dre_totais['CodigoUnidade'] == 0, 'CodigoUnidade'] = 9999111

# Validar unidades faltantes
unidades_faltantes = set([11112, 11114, 11115]) - set(dre_totais['CodigoUnidade'].unique())
print("Unidades faltantes:", unidades_faltantes)

# Consolidar novamente após correção
resumo_corrigido = dre_totais.groupby(['MesRef', 'CodigoUnidade'])['Valor'].sum().reset_index()
print(resumo_corrigido)


Unidades faltantes: {11112, 11114, 11115}
     MesRef  CodigoUnidade       Valor
0   2023-03          11111   112810.42
1   2023-03          11113   155887.64
2   2023-03          11116   332136.35
3   2023-03          11117  2226781.78
4   2023-03        9999111  3947137.85
5   2023-04          11111    56192.97
6   2023-04          11113   166398.78
7   2023-04          11116   391750.26
8   2023-04          11117  4215583.87
9   2023-04        9999111  6172323.18
10  2023-05          11111   214219.36
11  2023-05          11113   237780.94
12  2023-05          11116   577093.88
13  2023-05          11117  4917300.32
14  2023-05        9999111  9374836.62
15  2023-06          11111   113075.62
16  2023-06          11113   116338.98
17  2023-06          11116   410965.09
18  2023-06          11117  2969704.88
19  2023-06        9999111  5693319.49


In [39]:
# Verificar colunas das planilhas DRE originais
print("Colunas Maio 2023:", dre_maio.columns)
print("Colunas Junho 2023:", dre_junho.columns)

# Inspecionar dados por unidade
unidades_dre = pd.concat([dre_maio, dre_junho], axis=1)
print(unidades_dre.head())


Colunas Maio 2023: Index(['Nome', '9999111 - ESCRITÓRIO', 'Unidade 1 - 11111',
       'Unidade 2 - 11112 - Atacado', 'Unidade 3 - 11113',
       'Unidade 4 - 11114 - Atacado', 'Unidade 5 - 11115 - Atacado',
       'Unidade 6 - 11116', 'Unidade 7 - 11117', 'Total'],
      dtype='object')
Colunas Junho 2023: Index(['Nome', '9999111 - ESCRITÓRIO', 'Unidade 1 - 11111',
       'Unidade 2 - 11112 - Atacado', 'Unidade 3 - 11113',
       'Unidade 4 - 11114 - Atacado', 'Unidade 5 - 11115 - Atacado',
       'Unidade 6 - 11116', 'Unidade 7 - 11117', 'Total'],
      dtype='object')
                                        Nome  9999111 - ESCRITÓRIO  \
0              1 - Receita Operacional Bruta                   0.0   
1                1.1 - Vendas de mercadorias                   0.0   
2                1.2 - Prestação de Serviços                   0.0   
3  2 - Deduções da Receita Operacional Bruta                   0.0   
4     2.1 - Impostos Incidentes sobre Vendas                   0.0   

  

In [40]:
# Atualizar os códigos das unidades com base no nome
dre_totais['CodigoUnidade'] = dre_totais['Unidade'].replace({
    '9999111 - ESCRITÓRIO': 9999111,
    'Unidade 1 - 11111': 11111,
    'Unidade 2 - 11112 - Atacado': 11112,
    'Unidade 3 - 11113': 11113,
    'Unidade 4 - 11114 - Atacado': 11114,
    'Unidade 5 - 11115 - Atacado': 11115,
    'Unidade 6 - 11116': 11116,
    'Unidade 7 - 11117': 11117
})

# Revalidar os códigos faltantes
faltantes_pos_correcao = set(dim_unidade['CodigoUnidade']).difference(set(dre_totais['CodigoUnidade']))
print("Códigos faltantes após correção:", faltantes_pos_correcao)

# Consolidar valores por unidade e mês
dre_final = dre_totais.groupby(['MesRef', 'CodigoUnidade'])['Valor'].sum().reset_index()

# Visualizar resultado
print(dre_final)


Códigos faltantes após correção: set()
     MesRef CodigoUnidade       Valor
0   2023-03         11111   112810.42
1   2023-03         11112   213370.82
2   2023-03         11113   155887.64
3   2023-03         11114   394982.71
4   2023-03         11115   850746.17
5   2023-03         11116   332136.35
6   2023-03         11117  2226781.78
7   2023-03       9999111  -899338.87
8   2023-03         Total  3387377.02
9   2023-04         11111    56192.97
10  2023-04         11112   401397.39
11  2023-04         11113   166398.78
12  2023-04         11114   468650.09
13  2023-04         11115   567428.96
14  2023-04         11116   391750.26
15  2023-04         11117  4215583.87
16  2023-04       9999111  -766277.79
17  2023-04         Total  5501124.53
18  2023-05         11111   214219.36
19  2023-05         11112   440936.73
20  2023-05         11113   237780.94
21  2023-05         11114   613784.72
22  2023-05         11115  1585125.83
23  2023-05         11116   577093.88
24  2023-05

In [43]:
# Renomear 'IdUnidade' para 'CodigoUnidade' no fato_dre_final
fato_dre_final.rename(columns={'IdUnidade': 'CodigoUnidade'}, inplace=True)

# Verificar as colunas novamente
print("Colunas corrigidas no fato_dre_final:")
print(fato_dre_final.columns)


Colunas corrigidas no fato_dre_final:
Index(['IdEstrutura', 'CodigoUnidade', 'MesRef', 'Valor'], dtype='object')


In [44]:
# Comparar os valores do DRE com os do banco
comparacao = pd.merge(
    dre_final,
    fato_dre_final,
    on=['MesRef', 'CodigoUnidade'],
    how='outer',
    suffixes=('_dre', '_banco')
)

# Calcular diferenças
comparacao['Diferenca'] = comparacao['Valor_dre'] - comparacao['Valor_banco']
comparacao['Diferenca_Abs'] = comparacao['Diferenca'].abs()

# Exibir resultados ordenados
print(comparacao.sort_values(by=['MesRef', 'CodigoUnidade']))


        MesRef CodigoUnidade   Valor_dre  IdEstrutura  Valor_banco  Diferenca  \
0      2023-03         11111   112810.42          1.0     63205.51   49604.91   
1      2023-03         11111   112810.42          2.0     63205.51   49604.91   
2      2023-03         11111   112810.42          3.0         0.00  112810.42   
3      2023-03         11111   112810.42          4.0       -84.48  112894.90   
4      2023-03         11111   112810.42          5.0       -84.48  112894.90   
...        ...           ...         ...          ...          ...        ...   
37919  2023-06       9999111  -874322.83        162.0   -108316.28 -766006.55   
37920  2023-06       9999111  -874322.83        163.0   -108316.28 -766006.55   
37921  2023-06       9999111  -874322.83        162.0   -108316.28 -766006.55   
37922  2023-06       9999111  -874322.83        163.0   -108316.28 -766006.55   
37923  2023-06         Total  4651702.03          NaN          NaN        NaN   

       Diferenca_Abs  
0   

In [45]:
# Consolidar valores no banco
fato_consolidado = fato_dre_final.groupby(['MesRef', 'CodigoUnidade'])['Valor'].sum().reset_index()

# Renomear coluna para facilitar comparação
fato_consolidado.rename(columns={'Valor': 'Valor_banco'}, inplace=True)

# Verificar resultado
print(fato_consolidado.head())


    MesRef  CodigoUnidade  Valor_banco
0  2023-03          11111  -1648059.02
1  2023-03          11112   4788390.50
2  2023-03          11113   9202675.61
3  2023-03          11114  17572359.85
4  2023-03          11115  49717819.37


In [46]:
# Atualizar a comparação com valores consolidados
comparacao = pd.merge(
    dre_final,
    fato_consolidado,
    on=['MesRef', 'CodigoUnidade'],
    how='outer'
)

# Calcular diferenças
comparacao['Diferenca'] = comparacao['Valor'] - comparacao['Valor_banco']
comparacao['Diferenca_Abs'] = comparacao['Diferenca'].abs()

# Exibir resultado ordenado
print(comparacao.sort_values(by=['MesRef', 'CodigoUnidade']))


     MesRef CodigoUnidade       Valor   Valor_banco     Diferenca  \
0   2023-03         11111   112810.42 -1.648059e+06  1.760869e+06   
1   2023-03         11112   213370.82  4.788390e+06 -4.575020e+06   
2   2023-03         11113   155887.64  9.202676e+06 -9.046788e+06   
3   2023-03         11114   394982.71  1.757236e+07 -1.717738e+07   
4   2023-03         11115   850746.17  4.971782e+07 -4.886707e+07   
5   2023-03         11116   332136.35  2.192998e+07 -2.159784e+07   
6   2023-03         11117  2226781.78 -2.268858e+08  2.291126e+08   
7   2023-03       9999111  -899338.87 -1.155220e+08  1.146226e+08   
8   2023-03         Total  3387377.02           NaN           NaN   
9   2023-04         11111    56192.97 -5.487106e+06  5.543299e+06   
10  2023-04         11112   401397.39  2.116401e+07 -2.076261e+07   
11  2023-04         11113   166398.78  1.038250e+07 -1.021610e+07   
12  2023-04         11114   468650.09  2.727361e+07 -2.680496e+07   
13  2023-04         11115   567428

In [51]:
# Verificar duplicatas no banco com colunas corretas
duplicatas_banco = fato_dre_final[fato_dre_final.duplicated(
    subset=['MesRef', 'CodigoUnidade', 'IdEstrutura'],
    keep=False
)]

# Exibir duplicatas, se existirem
print("Duplicatas no banco:")
print(duplicatas_banco)


Duplicatas no banco:
       IdEstrutura  CodigoUnidade   MesRef      Valor
1288           162        9999111  2023-03 -112045.58
1289           163        9999111  2023-03 -112045.58
1290           162        9999111  2023-03 -112045.58
1291           163        9999111  2023-03 -112045.58
1292           162        9999111  2023-03 -112045.58
...            ...            ...      ...        ...
37915          163          11117  2023-06  106246.75
37916          162          11117  2023-06  106246.75
37917          163          11117  2023-06  106246.75
37918          162          11117  2023-06  106246.75
37919          163          11117  2023-06  106246.75

[32768 rows x 4 columns]


In [52]:
fato_dre_final = fato_dre_final.drop_duplicates(
    subset=['MesRef', 'CodigoUnidade', 'IdEstrutura'],
    keep='first'
)


In [53]:
duplicatas_banco = fato_dre_final[fato_dre_final.duplicated(
    subset=['MesRef', 'CodigoUnidade', 'IdEstrutura'],
    keep=False
)]

print("Duplicatas após limpeza:")
print(duplicatas_banco)


Duplicatas após limpeza:
Empty DataFrame
Columns: [IdEstrutura, CodigoUnidade, MesRef, Valor]
Index: []


In [54]:
# Recalcular as diferenças após limpeza
comparacao = pd.merge(
    dre_final,
    fato_dre_final,
    on=['MesRef', 'CodigoUnidade'],
    suffixes=('_dre', '_banco')
)

comparacao['Diferenca'] = comparacao['Valor_dre'] - comparacao['Valor_banco']
comparacao['Diferenca_Abs'] = comparacao['Diferenca'].abs()

# Mostrar resultados
print(comparacao)


       MesRef CodigoUnidade  Valor_dre  IdEstrutura  Valor_banco  Diferenca  \
0     2023-03         11111  112810.42            1     63205.51   49604.91   
1     2023-03         11111  112810.42            2     63205.51   49604.91   
2     2023-03         11111  112810.42            3         0.00  112810.42   
3     2023-03         11111  112810.42            4       -84.48  112894.90   
4     2023-03         11111  112810.42            5       -84.48  112894.90   
...       ...           ...        ...          ...          ...        ...   
5211  2023-06       9999111 -874322.83          159   -100107.53 -774215.30   
5212  2023-06       9999111 -874322.83          160     -8208.75 -866114.08   
5213  2023-06       9999111 -874322.83          161     -8208.75 -866114.08   
5214  2023-06       9999111 -874322.83          162   -108316.28 -766006.55   
5215  2023-06       9999111 -874322.83          163   -108316.28 -766006.55   

      Diferenca_Abs  
0          49604.91  
1      

In [55]:
# Consolidar valores no banco
banco_agrupado = fato_dre_final.groupby(['MesRef', 'CodigoUnidade'])['Valor'].sum().reset_index()
banco_agrupado.rename(columns={'Valor': 'Valor_banco'}, inplace=True)

# Recalcular comparação após consolidação
comparacao_final = pd.merge(
    dre_final,
    banco_agrupado,
    on=['MesRef', 'CodigoUnidade'],
    how='left'
)

comparacao_final['Diferenca'] = comparacao_final['Valor'] - comparacao_final['Valor_banco']
comparacao_final['Diferenca_Abs'] = comparacao_final['Diferenca'].abs()

# Mostrar resultados
print(comparacao_final)


     MesRef CodigoUnidade       Valor  Valor_banco  Diferenca  Diferenca_Abs
0   2023-03         11111   112810.42    111089.14    1721.28        1721.28
1   2023-03         11112   213370.82    217842.98   -4472.16        4472.16
2   2023-03         11113   155887.64    164731.03   -8843.39        8843.39
3   2023-03         11114   394982.71    411773.89  -16791.18       16791.18
4   2023-03         11115   850746.17    898514.57  -47768.40       47768.40
5   2023-03         11116   332136.35    353248.61  -21112.26       21112.26
6   2023-03         11117  2226781.78   2002820.33  223961.45      223961.45
7   2023-03       9999111  -899338.87  -1011384.45  112045.58      112045.58
8   2023-03         Total  3387377.02          NaN        NaN            NaN
9   2023-04         11111    56192.97     50774.30    5418.67        5418.67
10  2023-04         11112   401397.39    421693.20  -20295.81       20295.81
11  2023-04         11113   166398.78    176385.19   -9986.41        9986.41

In [56]:
import sqlite3
import pandas as pd

# Conectar ao banco SQLite
conn = sqlite3.connect('dre_tratado_final.db')

# Salvar as tabelas no banco
fato_dre_final.to_sql('fato_dre_final', conn, if_exists='replace', index=False)
dim_unidade.to_sql('dim_unidade', conn, if_exists='replace', index=False)
dim_estrutura.to_sql('dim_estrutura', conn, if_exists='replace', index=False)

# Fechar conexão
conn.close()

# Disponibilizar o arquivo para download
from google.colab import files
files.download('dre_tratado_final.db')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>